In [ ]:
import ROOT
# from T2KStyle import T2KStyle
c = ROOT.TCanvas('c', '', 800, 600)
cl = ROOT.TCanvas('cl', '', 1600, 1200)
cw = ROOT.TCanvas('c2', '', 1600, 600)

In [ ]:
# path = '~/LXPLUS/DESY_testbeam/tree/'
path = '/eos/user/s/ssuvorov/DESY_testbeam/cosmic_strict_v2/'
file_name = path + '/er3_380V'
N_iter = 4

In [ ]:
PRF_function  = ROOT.TF1("PRF_function",
                         "[0]*(1+[1]*x*x + [2] * x*x*x*x) / (1+[3]*x*x+[4]*x*x*x*x)",
                         -0.027, 0.027);
PRF_function.SetLineColor(ROOT.kRed)

file = ROOT.TFile(file_name + '_iter' + str(N_iter-1) + '.root')
cl.Clear()
cl.Divide(2, 2)
for i in range(4):
    if i == 0:
        histo = file.Get('PRF_histo')
        gr = file.Get('PRF_graph')
    else:
        histo = file.Get('prf_mult/PRF_histo_' + str(i+1) + 'pad')
        gr = file.Get('prf_mult/PRF_graph_' + str(i+1) + 'pad')
    cl.cd(i+1)
#     histo.Fit('PRF_function', 'Q', '', -0.025, 0.025)
    histo.GetYaxis().SetRangeUser(0., 1.)
    histo.Draw('colz')
#     gr.GetFunction("PRF_function").SetLineColor(ROOT.kRed)
    gr.SetLineColor(ROOT.kRed)
    gr.SetMarkerColor(ROOT.kRed)
    gr.Draw('same p')
cl.Draw()

# PRF over iterations

In [ ]:
par = []
par_col = []
for i in range(5):
    par.append(ROOT.TGraphErrors())
    par_col.append(ROOT.TGraphErrors())

for it in range(N_iter):
    file = ROOT.TFile(file_name + '_iter' + str(it) + '.root')
    func = file.Get('PRF_function') 
    for i in range(5):
        par[i].SetPoint(it, it, func.GetParameter(i))
        par[i].SetPointError(it, func.GetParError(i))

# PRF over column

In [ ]:
file = ROOT.TFile(file_name + '_iter' + str(2) + '.root')
for i in range(36):
    h = file.Get(f'prf_column/PRF_histo_col_{i}')
    h.Fit('PRF_function', 'Q', '', -0.025, 0.025)
    for j in range(5):
        par_col[j].SetPoint(i, i, PRF_function.GetParameter(j))
        par_col[j].SetPointError(i, PRF_function.GetParError(j))

In [ ]:
par_name = ['Const', 'a2', 'a4', 'b2', 'b4']
cl.Divide(2, 3)
for i in range(1, 6):
    c.cd(i)
    par[i-1].GetYaxis().SetTitle(par_name[i-1])
    par[i-1].GetXaxis().SetTitle('Iteration')
    par[i-1].Draw('ap')
    
cl.Draw()

In [ ]:
c = ROOT.TCanvas('c', '', 1600, 1200)

par_name = ['Const', 'a2', 'a4', 'b2', 'b4']
c.Divide(2, 3)
for i in range(1, 6):
    c.cd(i)
    par_col[i-1].GetYaxis().SetTitle(par_name[i-1])
    par_col[i-1].GetXaxis().SetTitle('Column')
    par_col[i-1].Draw('ap')
    
c.Draw()

# PRF MM vs ERAM

In [ ]:
N_iter = 5
file_mm = ROOT.TFile(path + 'mm1_380V_200ns_iter' + str(N_iter) + '.root', 'READ')
file_er = ROOT.TFile(path + 'er2_360V_iter' + str(N_iter) + '.root', 'READ')

PRF_er_hi = file_er.Get('PRF_histo')
PRF_mm_gr = file_mm.Get('PRF_graph')
PRF_er_gr = file_er.Get('PRF_graph')
PRF_mm_hi = file_mm.Get('PRF_histo')


c.Clear()
leg = ROOT.TLegend(0.2,0.2,0.4,0.4);

PRF_er_gr.GetFunction('PRF_function').SetLineColor(ROOT.kGreen)
PRF_mm_gr.GetFunction('PRF_function').SetLineColor(ROOT.kRed)
PRF_er_gr.GetFunction('PRF_function').GetYaxis().SetTitle('Q_{pad}/Q_{cluster}')
PRF_er_gr.GetFunction('PRF_function').GetXaxis().SetTitle('x_{pad}-x_{track}')
PRF_er_gr.GetFunction('PRF_function').GetYaxis().SetRangeUser(0., 1.)
PRF_er_gr.GetFunction('PRF_function').Draw('l')
PRF_mm_gr.GetFunction('PRF_function').Draw('same l')
leg.AddEntry(PRF_er_gr, 'ERAM2 360V', 'l')
leg.AddEntry(PRF_mm_gr, 'MM1 380V', 'l')
c.Draw()
# leg.Draw()

In [ ]:
cl.Clear()
cl.Divide(2, 2)

file_mm = ROOT.TFile(path + 'mm1_380V_200ns_iter' + str(0) + '.root', 'READ')
file_er = ROOT.TFile(path + 'er2_360V_iter' + str(0) + '.root', 'READ')

PRF_er_hi = file_er.Get('PRF_histo')
PRF_mm_hi = file_mm.Get('PRF_histo')
cl.cd(1)
PRF_mm_hi.GetYaxis().SetRangeUser(0., 1.)
PRF_mm_hi.Draw('colz')
cl.cd(3)
PRF_er_hi.GetYaxis().SetRangeUser(0., 1.)
PRF_er_hi.Draw('colz')


file_mm_last = ROOT.TFile(folder + 'mm1_380V_200ns_iter' + str(N_iter) + '.root', 'READ')
file_er_last = ROOT.TFile(folder + 'er2_360V_iter' + str(N_iter) + '.root', 'READ')

PRF_er_hi = file_er_last.Get('PRF_histo')
PRF_mm_hi = file_mm_last.Get('PRF_histo')

cl.cd(2)
PRF_mm_hi.GetYaxis().SetRangeUser(0., 1.)
PRF_mm_hi.Draw('colz')
cl.cd(4)
PRF_er_hi.GetYaxis().SetRangeUser(0., 1.)
PRF_er_hi.Draw('colz')

cl.Draw()
